In [12]:
import datetime
import pandas as pd

In [180]:
df = pd.read_csv("local/games.csv", parse_dates=["GAME_DATE_EST"], dtype={
    "HOME_TEAM_ID": str,
    "VISITOR_TEAM_ID": str,
    "GAME_ID": str,
    "SEASON": str,
    "HOME_TEAM_WINS": str
})

# Drop redundant columns and entries with null columns
df = df.drop(["GAME_STATUS_TEXT", "TEAM_ID_home", "TEAM_ID_away"], axis=1)
df = df.dropna()

# Sort by ascending date
df = df.sort_values("GAME_DATE_EST")

# Set game date as index
# df = df.set_index("GAME_DATE_EST", drop=False)

df.head()

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
19288,2003-10-05,10300001,1610612762,1610612742,2003,90.0,0.457,0.735,0.143,23.0,41.0,85.0,0.447,0.500,0.250,20.0,38.0,1
19287,2003-10-06,10300002,1610612763,1610612749,2003,105.0,0.494,0.618,0.267,25.0,48.0,94.0,0.427,0.700,0.154,20.0,43.0,1
19280,2003-10-07,10300010,1610612764,1610612752,2003,104.0,0.506,0.677,0.455,26.0,45.0,86.0,0.380,0.852,0.188,19.0,37.0,1
19286,2003-10-07,10300009,1610612758,1610612746,2003,101.0,0.467,0.871,0.444,19.0,39.0,82.0,0.368,0.609,0.364,13.0,50.0,1
19285,2003-10-07,10300005,1610612757,1610612745,2003,104.0,0.527,0.657,0.429,22.0,33.0,80.0,0.470,0.667,0.333,10.0,37.0,1


In [183]:
# # Filter only columns that may be used during prediction
# df_training = df[["GAME_DATE_EST", "HOME_TEAM_ID", "VISITOR_TEAM_ID", "SEASON", "HOME_TEAM_WINS"]]

df_aug = df.copy()
df_aug[[
    "prev_PTS_home",
    "prev_FG_PCT_home",
    "prev_FT_PCT_home",
    "prev_FG3_PCT_home",
    "prev_AST_home",
    "prev_REB_home"
]] = df.groupby(["HOME_TEAM_ID"]).shift(1)[[
    "PTS_home",
    "FG_PCT_home",
    "FT_PCT_home",
    "FG3_PCT_home",
    "AST_home",
    "REB_home"
]]
df_aug = df_aug.dropna()
df_aug = df_aug[[
    "GAME_DATE_EST",
    "HOME_TEAM_ID",
    "VISITOR_TEAM_ID",
    "SEASON",
    "prev_PTS_home",
    "prev_FG_PCT_home",
    "prev_FT_PCT_home",
    "prev_FG3_PCT_home",
    "prev_AST_home",
    "prev_REB_home",
    "HOME_TEAM_WINS"
]]
df_aug.head()

,GAME_DATE_EST,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,prev_PTS_home,prev_FG_PCT_home,prev_FT_PCT_home,prev_FG3_PCT_home,prev_AST_home,prev_REB_home,HOME_TEAM_WINS
19164,2003-10-29,1610612765,1610612754,2003,96.0,0.391,0.600,0.444,19.0,37.0,0
19162,2003-10-29,1610612738,1610612748,2003,89.0,0.462,0.538,0.214,21.0,36.0,1
19167,2003-10-29,1610612758,1610612739,2003,101.0,0.467,0.871,0.444,19.0,39.0,1
19169,2003-10-29,1610612761,1610612751,2003,76.0,0.383,0.750,0.200,17.0,39.0,1
19170,2003-10-29,1610612740,1610612737,2003,105.0,0.527,0.714,0.700,27.0,39.0,1


In [178]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

le = LabelEncoder()
df = df_aug_training.copy()
df[[
    "HOME_TEAM_ID",
    "VISITOR_TEAM_ID",
    "SEASON"
]] = df[[
    "HOME_TEAM_ID",
    "VISITOR_TEAM_ID",
    "SEASON"
]].apply(le.fit_transform)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26492 entries, 19152 to 0
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   GAME_DATE_EST      26492 non-null  datetime64[ns]
 1   HOME_TEAM_ID       26492 non-null  int64         
 2   VISITOR_TEAM_ID    26492 non-null  int64         
 3   SEASON             26492 non-null  int64         
 4   prev_PTS_home      26492 non-null  float64       
 5   prev_FG_PCT_home   26492 non-null  float64       
 6   prev_FT_PCT_home   26492 non-null  float64       
 7   prev_FG3_PCT_home  26492 non-null  float64       
 8   prev_AST_home      26492 non-null  float64       
 9   prev_REB_home      26492 non-null  float64       
 10  HOME_TEAM_WINS     26492 non-null  object        
dtypes: datetime64[ns](1), float64(6), int64(3), object(1)
memory usage: 2.4+ MB
